### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [257]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1
 
    def __init__(self, min_samples_split=2, min_samples_leaf=1, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise
 
        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise
 
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - ((np.sum(np.hstack((l_c * l_c / l_s, r_c * r_c / r_s)),
                            axis=1)) / (l_s[0] + r_s[0]))

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum(np.hstack((l_c * np.log2(l_c / l_s),
                                  r_c * np.log2(r_c / r_s)),
                                 axis=1))/(l_s[0] + r_s[0])

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (np.max(l_c, axis=1) + np.max(r_c, axis=1)) / (l_s + r_s)
    
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]
 
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]
 
    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
 
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]
    
    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # сортировка x и y
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # class_number - количество уникальных классов в y
        class_number = self.num_class
 
        # cut_size - у нас точно граница пройдет не через них
        cut_size = round(self.min_samples_split / 2) - 1
        # делим наши y 
        if (cut_size != 0):
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        else:
            splitted_sorted_y = sorted_y
        # здесь находим индексы элементов, у которых сосед совпадает с ними
        # то есть находим правых соседей пар одинаковых элементов
        # сохраняем в r_border_ids
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (cut_size + 1)
        
        # если класс не менялся вообще, выходим из threshold
        if len(r_border_ids) == 0:
            return float('+inf'), None
 
        # Получаем количество одинаковых идущих подряд элементов в интервале.
        # Потом создаем матрицу, с единицами на местах где происходит смена.
        eq_el_count = r_border_ids - np.append([cut_size], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        # Затем делаем матрицу количества классов для каждого интервала.
        # Для 0 позиции добавляем число вырезанных элементов. 
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:cut_size], minlength=class_number)
 
        # В i строке l_class_count будут лежать количества
        # элементов каждого класса встретившихся до r_border_ids[i]-го момента
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y, minlength=class_number) - l_class_count
        # Размеры разбиений
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes
 
        # Считаем Gain, берем наименьший
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
 
        # left_el_id - индекс наилучшего разбиения
        left_el_id = l_sizes[idx][0]
        # Возвращает эту меру неопределенности и элемент и значение, для разбиения выборки
        return gs[idx], (sorted_x[left_el_id - 1] + sorted_x[left_el_id]) / 2.0
 
 
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)
        return self


    def __fit_node(self, x, y, node_id, depth, ind=-1):
        # ind = -1 чтобы отследить факт захода первый
        # ввел так как нельзя менять fit
        if (ind == -1):
            self.feature_importances_ = np.zeros(x.shape[1])
            self.Size_ = x.shape[1]    
        # p - вероятности
        p = np.bincount(y, minlength=self.num_class) / x.shape[0]
        moda = np.argmax(p)
        # Критерии останова
        if self.max_depth is not None and depth >= self.max_depth or\
                    y.shape[0] < self.min_samples_split or\
                    self.sufficient_share <= p[moda]:
            self.tree[node_id] = (self.LEAF_TYPE, moda, p)
            return 

        ids = self.get_feature_ids(x.shape[1])
        thresholds = np.array([self.__find_threshold(x[:, idx], y) for idx in ids])
        # best - наилучший признак, по какому бить надо
        best = thresholds[:, 0].argmin()
        thr = thresholds[best, 1]
        # imp_curr - новая неопределенность, самая наименьшая по фичам
        imp_curr = thresholds[best, 0]

        if (thr is None):
            self.tree[node_id] = (self.LEAF_TYPE, moda, p)
            return

        x_l, x_r, y_l, y_r = self.__div_samples(x, y, ids[best], thr)
        if (x_l.shape[0] == 0 or x_r.shape[0] == 0):
            self.tree[node_id] = (self.LEAF_TYPE, moda, p)
            return

        self.tree[node_id] = (self.NON_LEAF_TYPE, ids[best], thr)

        # считаем feature_importance
        # imp_prev - значение прошлой неопределенности в узле
        if self.G_function == self.__gini:
            imp_prev = 1 - np.sum(p ** 2)
        elif self.G_function == self.__entropy:
            imp_prev = -np.sum(p * np.log2(p))
        else:
            imp_prev = 1 - np.max(p)
        # print('imp_prev = ', imp_prev, ' '*5, 'imp_curr = ', imp_curr)
        self.feature_importances_[ids[best]] += x.shape[0] * (imp_prev - imp_curr) / self.Size_
        
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1, 0)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1, 0)

        
        
    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]
 
    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
 
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
 
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])
 
    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [268]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [269]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [270]:
%time clf.fit(X_train, y_train)

CPU times: user 2.62 ms, sys: 0 ns, total: 2.62 ms
Wall time: 1.78 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [271]:
%time my_clf.fit(X_train, y_train)

CPU times: user 25 ms, sys: 53 µs, total: 25.1 ms
Wall time: 24.3 ms


## Проверка качества работы на wine

In [272]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9474747474747475

In [273]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9474747474747475

## Подготовка данных Speed Dating Data 

In [274]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', encoding='cp1251', low_memory = False)

In [275]:
df = df.iloc[:, :97]
df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1',
             'order', 'partner', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin',
              'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'dec_o',
              'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o',
              'like_o', 'prob_o', 'met_o', 'field', 'undergra',
              'imprelig', 'imprace', 'from', 'zipcode', 'career', 'sports',
              'tvsports', 'exercise', 'dining', 'museums',
              'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv',
              'theater', 'movies', 'concerts', 'music', 'shopping',
              'yoga', 'expnum', 'wave'], axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = pd.get_dummies(df, columns=['field_cd'],
                    prefix='field_cd', prefix_sep='=')
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.mn_sat = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.tuition = df.tuition.fillna(-999)
#df = pd.get_dummies(df, columns=['race'],
#                    prefix='race', prefix_sep='=')
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])

df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = pd.get_dummies(df, columns=['career_c'],
                        prefix='career_c', prefix_sep='=')

a = ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
df.loc[:, 'temp_totalsum'] = df.loc[:, a].sum(axis=1)
df.loc[:, a] =\
    (df.loc[:, a].T / df.loc[:, 'temp_totalsum'].T).T * 100
a = ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
df.loc[:, 'temp_totalsum'] = df.loc[:, a].sum(axis=1)
df.loc[:, a] =\
    (df.loc[:, a].T / df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
if i != 4:
    feat.remove('shar{}_1'.format(i))
    
df = df.drop(feat, axis=1)

df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values


## Проверка скорости работы на Speed Dating Data 

In [287]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [288]:
%time clf.fit(X_train, y_train)

CPU times: user 85.2 ms, sys: 7.61 ms, total: 92.8 ms
Wall time: 91.2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [289]:
%time my_clf.fit(X_train, y_train)

CPU times: user 502 ms, sys: 0 ns, total: 502 ms
Wall time: 503 ms


## Проверка качества работы на Speed Dating Data

In [290]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5358670541769134

In [291]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.45383275261324035

## Задание 3

## Задание 4

In [292]:
data = my_clf.feature_importances_ / my_clf.feature_importances_.sum()
My_feature = pd.Series(data=data, index=df_pair.columns[1:])

data_ = clf.feature_importances_
My_feature_ = pd.Series(data=data_, index=df_pair.columns[1:])

print(My_feature.sort_values(ascending = False)[:10],'\n', My_feature_.sort_values(ascending = False)[:10], sep='')

field_cd=10.0_f    0.124595
attr2_1            0.109317
shar1_1_f          0.101267
race_f             0.096836
attr3_1            0.096658
sinc1_1            0.091187
shar1_1            0.076101
date               0.075152
go_out_f           0.068485
mn_sat             0.056364
dtype: float64
int_corr      0.074165
income_f      0.030525
amb2_1        0.030185
exphappy_f    0.024220
amb4_1_f      0.023875
date          0.022936
attr3_1_f     0.022295
fun1_1        0.021568
shar4_1       0.021214
amb3_1_f      0.020402
dtype: float64


## Задание 5

In [293]:
param_grid = {
    'max_depth': [i for i in range(1, 10)],
    'max_features': ['auto','sqrt','log2'], 
    'min_impurity_decrease': [i / 1000 for i in range(1, 20)],
    "n_estimators": [i for i in range(5, 100, 5)],
} 

model = RandomForestClassifier(random_state=123)
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=100, n_jobs=-1,
                                   scoring='roc_auc', random_state=123)
random_search.fit(X, y)
print(random_search.best_params_)

/home/ruslanallayarov/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'n_estimators': 80, 'min_impurity_decrease': 0.001, 'max_features': 'log2', 'max_depth': 6}
